In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# Base model and adapter paths
base_model = "meta-llama/Llama-3.1-8B-Instruct"
adapter_path = "./kto_adapter"  # path to your trained adapter

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(adapter_path)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    dtype=torch.bfloat16
)

# Load adapter into the model
model = PeftModel.from_pretrained(model, adapter_path)
model.eval()



/home/sujayb/miniconda3/envs/sujay_fix/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.46it/s]


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [2]:


# Create a prompt
prompt = "What is the capital of France?"

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)


In [ ]:
gen_conf = {
    remove_nan
}

{'input_ids': tensor([[128000,   3923,    374,    279,   6864,    315,   9822,     30]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:

# Generate output
with torch.no_grad():
    out = model(**inputs, generation_config=gen_conf)
    print(out)
    logits = out.logits
    if torch.isnan(logits).any():
        print("⚠️ Found NaN in logits!")
        # replace nans with -ve inf
        logits = torch.where(torch.isnan(logits), torch.full_like(logits, float('-inf')), logits)

    if torch.isinf(logits).any():
        print("⚠️ Found Inf in logits!")
        logits = torch.where(torch.isinf(logits), torch.full_like(logits, float('-inf')), logits)

    # Get the predicted token
    predicted_token_id = torch.argmax(logits[:, -1, :], dim=-1)
    answer = tokenizer.decode(predicted_token_id)


# Decode and print the model's answer
print(answer)

CausalLMOutputWithPast(loss=None, logits=tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., nan, nan, nan]]], device='cuda:0',
       dtype=torch.bfloat16), past_key_values=DynamicCache(layers=[DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer]), hidden_states=None, attentions=None)
⚠️ Found NaN in logits!
⚠️ Found Inf in logits!
!
